In [1]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.layouts import widgetbox, column
output_notebook()

Loading BokehJS ...

In [176]:
def telaraña(f, x, xiniciales, nmax=200):
    "Dibuja el diagrama de telaraña de f sobre el arreglo x, con valor(es) inicial(es) xinic."
    xiniciales = np.array([xiniciales]).flatten()
    "grafica la función sobre x y la recta identidad."
    p=figure()
    p.line(x, f(x), color='black', line_width=3)
    p.line(x, x, color="red", line_width=3)
    "iteraciones"
    n=xiniciales.size
    x1 = np.empty((n, nmax//2))
    x1[:,0] = xiniciales
    _f = np.frompyfunc(lambda x,_: f(x), 2, 1) # accumulate requiere función binaria pero f se asume unaria
    x2 = _f.accumulate(x1, dtype=np.object, axis=1).astype(np.float)
    x3 = np.repeat(x2, 2, axis=1)
    "construyendo y"
    y1 = f(x3)
    y3 = np.concatenate((np.array([[0]]*n), y1[:,:-1]), axis=1)
    source = ColumnDataSource(data=dict(x=x3, y=y3))
    callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var n = cb_obj.value
    xs = data['x']
    ys = data['y']

    /*for (i = 0; i < xs.length; i++) {
        data['x'].push(xs[i].slide(0,n))
        data['y'].push(ys[i].slide(0,n))
    }*/
    data['x'] = xs.slice(0,n)
    data['y'] = ys.slice(0,n)
    source.change.emit();
    data['x'] = xs;
    data['y'] = ys;
    """)
    sl = Slider(start=1, end=nmax, value=1, step=1, title="Pasos")
    sl.js_on_change('value', callback)
    p.line('x', 'y', source=source, color=tuple(np.random.randint(0, 256, 3)))
    show(column([widgetbox(sl), p]))

In [290]:
def telaraña2(f, x, xiniciales, nmax=200):
    "Dibuja el diagrama de telaraña de f sobre el arreglo x, con valor(es) inicial(es) xinic."
    xiniciales = np.array([xiniciales, xiniciales]).flatten()
    "grafica la función sobre x y la recta identidad."
    p=figure()
    p.line(x, f(x), color='black', line_width=3)
    p.line(x, x, color="red", line_width=3)
    "iteraciones"
    n=len(xiniciales)
    x1 = np.empty((n, nmax//2))
    x1[:,0] = xiniciales
    _f = np.frompyfunc(lambda x,_: f(x), 2, 1) # accumulate requiere función binaria pero f se asume unaria
    x2 = _f.accumulate(x1, dtype=np.object, axis=1).astype(np.float)
    x3 = np.repeat(x2, 2, axis=1)
    "construyendo y"
    y1 = f(x3)
    y3 = np.concatenate((np.array([[0]]*n), y1[:,:-1]), axis=1)
    source = ColumnDataSource(data=dict(x=x3[0], y=y3[0], x0=x3[1], y0=y3[1]))
    def callback(source=source, window=None):
        data = source.data
        nv = cb_obj.value
        xs = data['x']
        ys = data['y']
        data['x'] = xs[:nv]
        data['y'] = ys[:nv]
        x0s = data['x0']
        y0s = data['y0']
        data['x0'] = x0s[:nv]
        data['y0'] = y0s[:nv]
        source.change.emit()
        data['x'] = xs
        data['y'] = ys
        data['x0'] = x0s
        data['y0'] = y0s
    sl = Slider(start=1, end=nmax, value=1, step=1, title="Pasos", callback=CustomJS.from_py_func(callback))
    #sl.js_on_change('value', callback)
    p.line('x', 'y', source=source, color=tuple(np.random.randint(0, 256, 3)))
    if n>1:
        p.line('x0', 'y0', source=source, color=tuple(np.random.randint(0, 256, 3)))
    show(column([widgetbox(sl), p]))

In [294]:
def logística(x, r=4):
    return r * x*(1-x)

x = np.arange(0, 1, .01)

%time telaraña2(logística, x, [.4,.39999])

CPU times: user 673 ms, sys: 6.31 ms, total: 679 ms
Wall time: 676 ms


In [295]:
telaraña2(lambda x:x/2, np.arange(-1,1,.01), [-1, 1])

In [233]:
a=np.empty((1,10))
a[:,0]=[3]
a

array([[3.0e+000, 2.0e+000, 4.4e-323, 0.0e+000, 0.0e+000, 0.0e+000,
        0.0e+000, 0.0e+000, 0.0e+000, 0.0e+000]])